# Truncating an MPS

## Algorithm

The "simplification" algorithm consists on approximating a quantum state $|\Psi\rangle,$ encoded in a large bond dimension MPS, using an MPS with a smaller bond dimension fixed bond dimension $\xi$, $|\phi\rangle\in\mathrm{MPS}_\xi$. We work by minimizing the distance between both states
$$\phi = \mathrm{argmin}_{\phi\in \mathrm{MPS}_\xi} d(\Psi,\phi).$$

The algorithm uses this distance expressed as a sum of scalar products
$$d(\Psi,\phi) = \|\Psi-\phi\|^2 = \langle\Psi|\Psi\rangle + \langle\phi|\phi\rangle - \langle\Psi|\phi\rangle - \langle\phi|\Psi\rangle$$

Given the structure of scalar products we see that the distance is a bilinear function with respect to any of the tensors in $|\phi\rangle.$ Take for instance an MPS with four sites and let us focus on the second site. The scalar product between $\Psi$ and $\phi$ can be written as the contraction between two environments and two tensors from each state

<img src="figures/scalar-product-with-environments.svg" style="max-width:90%; width:30em">

As explained in [this notebook](File%201c%20-%20Canonical%20form.ipynb), if $|\phi\rangle$ is in canonical form with respect to that site., its norm can be written directly as the contraction of the same tensor and its complex conjugate.

<img src="figures/scalar-product-canonical.svg" style="max-width:90%; width:30em">

This structure allows us to write the condition for optimality with respect to a site
$$\frac{\partial}{\partial D^{i}_{\alpha\beta}} d(\Psi,\phi) = 0$$
as a linear equation with respect to the tensor
$$U^{i}_{\alpha\beta} - D^{i}_{\alpha\beta} = 0$$
with the elements
$$\frac{\partial}{\partial D^{i *}_{\alpha\beta}} \langle\phi|\Psi\rangle = U^{i}_{\alpha\beta}$$
and
$$\frac{\partial}{\partial D^{i *}_{\alpha\beta}} \langle\phi|\phi\rangle = D^{i}_{\alpha\beta}$$

### Optimization 1: linear form

As mentioned above, once our state $|\psi\rangle$ is in canonical form with respect to a given site, the solution to the problem is
$$D^i_{\alpha\beta} = U^i_{\alpha\beta}$$
where $U$ is constructed out of the left and right environments $L$ and $R$ and the tensor $C$.

Our algorithm goes a step further. We realize that $U$ is actually the representaiton of the antilinear form $\mathcal{L}$. If we define
$$\mathcal{L}[|\phi\rangle] := \langle\phi|\Psi\rangle,$$
this linear form is a linear function of the tensor $D$ given by
$$\mathcal{L}[|\phi\rangle] = \sum_{i\alpha\beta} \sum U^{i}_{\alpha\beta} D^{i *}_{\alpha\beta}.$$

Our algorithm therefore revolves around creating an object type called `AntilinearForm` that takes two states $|\Psi\rangle$ and $|\phi\rangle,$ one of them in canonical form around the $n$-th site, and returns the tensor $C$ that represents the linear form with respect to that site.

### Optimization 2: two-site method

The algorithm in this form has a drawback: *we need to know the size of the tensor* $D^i_{\alpha\beta}$ in advance. An useful variant is to optimize with respect to two sites, for instance $D$ and $F$ simultaneously, combining them in a single tensor that satisfies a similar equation. The two-site tensor is then split optimally, distributing the entanglement according to the maximum bond dimension that is allowed.

<img src="figures/two-site-optimization.svg" style="max-width:70%; width: 35em">

## Antilinear form

As anticipated above, the core of the algorithm is to engineer a `LinearForm` class that keeps track of the environments and tensors---$L$, $C$ and $R$--which are needed to reconstruct the linear form from one or two sites.

It is important to remark that the functions construting the tensors, `tensor1site()` and `tensor2sites()`, are deeply tied to the implementation of environments in [notebook 2a](File%202a%20-%20Expectation%20values.ipynb). We copy here those conventions in graphical form

<img src="figures/linear-form-contraction-order.svg" style="max-width:90%; width:30em">

In [ ]:
# file: mps/truncate.py

import numpy as np
import mps
from mps.state import *

from mps.expectation import \
    begin_environment, \
    update_right_environment, \
    update_left_environment

class AntilinearForm:
    #
    # This class is an object that formally implements <ψ|ϕ> with an argument
    # ϕ that may change and be updated over time.
    #
    # Given a site 'n' it returns the tensor 'L' such that the contraction
    # between 'L' and ϕ[n] is the result of the linear form."""
    #
    def __init__(self, ψ, ϕ, center=0):
        #
        # At the beginning, we create the right- and left- environments for
        # all sites to the left and to the right of 'center', which is the
        # focus point of 'LinearForm'.
        #
        ρ = begin_environment()
        R = [ρ] * ψ.size
        for i in range(ψ.size-1, center, -1):
            R[i-1] = ρ = update_right_environment(ϕ[i], ψ[i], ρ)

        ρ = begin_environment()
        L = [ρ] * ψ.size
        for i in range(0, center):
            L[i+1] = ρ = update_left_environment(ϕ[i], ψ[i], ρ)

        self.ψ = ψ
        self.ϕ = ϕ
        self.R = R
        self.L = L
        self.center = center

    def tensor1site(self):
        #
        # Return the tensor that represents the LinearForm at the 'center'
        # site of the MPS
        #
        center = self.center
        L = self.L[center]
        R = self.R[center]
        C = self.ψ[center]
        
        return np.einsum('li,ijk,kn->ljn', L, C, R)

    def tensor2site(self, direction):
        #
        # Return the tensor that represents the LinearForm using 'center'
        # and 'center+/-1'
        #
        center = self.center
        if direction > 0:
            i = self.center
            j = i + 1
        else:
            j = self.center
            i = j - 1
        L = self.L[i]
        A = self.ψ[i]
        B = self.ψ[j]
        R = self.R[j]
        LA = np.einsum('li,ijk->ljk', L, A)
        BR = np.einsum('kmn,no->kmo', B, R)
        return np.einsum('ljk,kmo->ljmo', LA, BR)

    def update(self, direction):
        #
        # We have updated 'ϕ', which is now centered on a different point.
        # We have to recompute the environments.
        #
        center = self.center
        if direction > 0:
            L = self.L[center]
            if center+1 < ψ.size:
                L = update_left_environment(ψ[center], ϕ[center], L)
                self.L[center+1] = L
                self.center = center+1
        else:
            R = self.R[center]
            if center > 0:
                R = update_right_environment(ψ[center], ϕ[center], L)
                self.R[center-1] = R
                self.center = center-1

## Two-site simplification loop

In [ ]:
# file: mps/truncate.py

def simplify_mps_2site(ψ, dimension=0, sweeps=2, direction=+1,
                       tolerance=DEFAULT_TOLERANCE):
    """Simplify an MPS ψ transforming it into another one with a smaller bond
    dimension, sweeping until convergence is achieved.
    
    Arguments:
    ----------
    sweeps = maximum number of sweeps to run
    tolerance = relative tolerance when splitting the tensors
    dimension = maximum bond dimension, 0 to just truncate to tolerance
    """
    
    if dimension == 0:
        return CanonicalMPS(ψ, center=0, tolerance=tolerance)

---

# Tests

In [ ]:
# file: mps/test/test_truncate.py

import unittest
from mps.test.tools import *

class TestLinearForm(unittest.TestCase):
    
    def test_canonical_env(self):
        #
        # When we pass two identical canonical form MPS to LinearForm, the
        # left and right environments are the identity
        #
        def ok(ψ):
            for center in range(ψ.size):
                ϕ = CanonicalMPS(ψ, center)
                LF = AntilinearForm(ϕ, ϕ, center)
                self.assertTrue(almostIdentity(LF.L[center],+1))
                self.assertTrue(almostIdentity(LF.R[center],+1))
        
        test_over_random_mps(ok)
    
    def tensor1siteok(self, aϕ, O):
        for center in range(aϕ.size):
            ϕ = CanonicalMPS(aϕ, center, normalize=True)
            for n in range(ϕ.size):
                #
                # Take an MPS Φ, construct a new state ψ = O1*ϕ with a local
                # operator on the 'n-th' site
                #
                ψ = ϕ.copy()
                ψ[n] = np.einsum('ij,ajb->aib', O, ψ[n])
                #
                # and make sure that the AntilinearForm provides the right tensor to
                # compute <ϕ|ψ> = <ϕ|O1|ϕ>
                #
                Odesired = ϕ.expectation1(O, n)
                LF = AntilinearForm(ψ, ϕ, ϕ.center)
                Oestimate = np.einsum('aib,aib', ϕ[ϕ.center], LF.tensor1site())
                self.assertAlmostEqual(Oestimate, Odesired)
                if n >= center:
                    self.assertTrue(almostIdentity(LF.L[center],+1))
                if n <= center:
                    self.assertTrue(almostIdentity(LF.R[center],+1))

    def test_tensor1site_product(self):
        O = np.array([[0.3,0.2+1.0j],[0.2-1.0j,2.0]])
        test_over_random_mps(lambda ϕ: self.tensor1siteok(ϕ, O), D=1)

    def test_tensor1site(self):
        O = np.array([[0.3,0.2+1.0j],[0.2-1.0j,2.0]])
        test_over_random_mps(lambda ϕ: self.tensor1siteok(ϕ, O))
    
    def tensor2siteok(self, aϕ, O1, O2):
        for center in range(aϕ.size):
            ϕ = CanonicalMPS(aϕ, center, normalize=True)
            for n in range(ϕ.size-1):
                #
                # Take an MPS Φ, construct a new state ψ = O1*ϕ with a local
                # operator on the 'n-th' site
                #
                ψ = ϕ.copy()
                ψ[n] = np.einsum('ij,ajb->aib', O1, ψ[n])
                ψ[n+1] = np.einsum('ij,ajb->aib', O2, ψ[n+1])
                #
                # and make sure that the AntilinearForm provides the right tensor to
                # compute <ϕ|ψ> = <ϕ|O1|ϕ>
                #
                Odesired = ϕ.expectation2(O1, O2, n)
                LF = AntilinearForm(ψ, ϕ, center)
                if center+1 < ϕ.size:
                    D = np.einsum('ijk,klm->ijlm', ϕ[center], ϕ[center+1])
                    Oestimate = np.einsum('aijb,aijb', D, LF.tensor2site(+1))
                    self.assertAlmostEqual(Oestimate, Odesired)
                if center > 0:
                    D = np.einsum('ijk,klm->ijlm', ϕ[center-1], ϕ[center])
                    Oestimate = np.einsum('aijb,aijb', D, LF.tensor2site(-1))
                    self.assertAlmostEqual(Oestimate, Odesired)
                if n >= center:
                    self.assertTrue(almostIdentity(LF.L[center],+1))
                if n+1 <= center:
                    self.assertTrue(almostIdentity(LF.R[center],+1))

    def test_tensor2site_product(self):
        O1 = np.array([[0.3,0.2+1.0j],[0.2-1.0j,2.0]])
        O2 = np.array([[0.34,0.4-0.7j],[0.4+0.7j,-0.6]])
        test_over_random_mps(lambda ϕ: self.tensor2siteok(ϕ, O1, O2), D=1)

    def test_tensor2site(self):
        O1 = np.array([[0.3,0.2+1.0j],[0.2-1.0j,2.0]])
        O2 = np.array([[0.34,0.4-0.7j],[0.4+0.7j,-0.6]])
        test_over_random_mps(lambda ϕ: self.tensor2siteok(ϕ, O1, O2))

In [ ]:
suite1 = unittest.TestLoader().loadTestsFromNames(['__main__.TestLinearForm'])
unittest.TextTestRunner(verbosity=2).run(suite1);